# OpenStreetMap Project of Austin, TX

##  Map Area

### Austin, TX

https://www.openstreetmap.org/node/1801308037#map=11/30.2715/-97.7436

I choose Austin, TX because it's the next travelling city I plan to visit. However, although I only looked for city of Austin, the osm file downloaded has included cities around Austin. The whole file looks too big to run locally in my computer. So I decide to extract a sample from original one by using code given on Project Detail page, and name it "Austin_split.osm'. Then I extract a smaller sample from 'Austin_split.osm' and name it "sample.osm'.

In [1]:
from collections import defaultdict
import xml.etree.ElementTree as ET
import pprint
import re
import sqlite3
import csv
from pprint import pprint
import os
from hurry.filesize import size

## Problems found in the OSM dataset

### 1. The values like 'atm' and 'bench' are the value of 'k' attribute, and also could be the value of 'v' attribute when value of 'k' attribute is 'amenity'.

In [2]:
amenity_set = defaultdict(set) #it will show that "atm" and "bench" could be either keys of dict or values of key

for _,element in ET.iterparse("Austin_split.osm", events = ("start",)):
    if element.tag == "node":
        for tag in element.iter("tag"):
            #make tag.attrib['k'] be the keys in dict, and tag.attrib['v'] be the value of key
            if tag.attrib['k'] == 'atm' or tag.attrib['k'] == 'bench' or tag.attrib['v'] == 'atm' or tag.attrib['v'] == 'bench':
                amenity_set[tag.attrib['k']].add(tag.attrib['v'])
            
amenity_set 

defaultdict(set,
            {'amenity': {'atm', 'bench'},
             'atm': {'yes'},
             'bench': {'no', 'yes'}})

### 2. The city names in "node" are not in consistent format, like 'Austin' and 'Austin, TX', 'Dripping Springs' and 'Dripping Springs, Tx'.

In [3]:
city_name_set = set() #to show all non-repeated city names

for _,element in ET.iterparse("Austin_split.osm", events = ("start",)):
    if element.tag == "node":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == 'addr:city':
                city_name_set.add(tag.attrib['v'])
                
city_name_set

{'Austin',
 'Austin, TX',
 'Buda',
 'Cedar Park',
 'Creedmoor',
 'Dripping Springs',
 'Dripping Springs, Tx',
 'Lakeway',
 'Manchaca',
 'Pflugerville'}

### 3. The street names are sometimes in abbreviation formats, like 'S 1st St, Suite 104', 'N HWY 183' and 'FM 535'.

### 4. 'trigger:zip_left' and 'tiger:zip_left_1' are not same in some cases, for example:
    
       <tag k="tiger:zip_left" v="78617" />
       <tag k="tiger:zip_left_1" v="78612" />
       <tag k="tiger:zip_left_2" v="78617" />
       <tag k="tiger:zip_right" v="78617" />
       <tag k="tiger:zip_right_1" v="78612" />
       <tag k="tiger:zip_right_2" v="78617" />

### 5. Some of the street name is repeated and divided into several segments, for example:
     
       <tag k="highway" v="residential" />
       <tag k="name" v="East 22nd Street" />
       <tag k="surface" v="asphalt" />
       <tag k="tiger:cfcc" v="A41" />
       <tag k="tiger:county" v="Travis, TX" />
       <tag k="tiger:name_base" v="22nd" />
       <tag k="tiger:name_direction_prefix" v="E" />
       <tag k="tiger:name_type" v="St" />
       <tag k="tiger:reviewed" v="no" />
       <tag k="tiger:zip_left" v="78722" />
       <tag k="tiger:zip_right" v="78722" />

### 6. The 'name' and 'name_1' in the 'tag' from same 'way' tag are not same in some cases like: 

       <tag k="name" v="Lakewood Drive" />
       <tag k="name_1" v="County Road 307" />

## Revise of Problem 2 about city names

I will unify city names under both "node" and "way" in the format of "CityName". That is applied in "Austin_split.osm".

In [4]:
#keys are inconsistant city names, values are right ones
mapping = { "Austin, TX": "Austin",
            "Austin, Tx": "Austin",
            "Dripping Springs, Tx": "Dripping Springs",
            "Pflugerville, TX": "Pflugerville", 
            "Westlake Hills, TX": "West Lake Hills",
            "austin": "Austin",
            "Ste 128, Austin": "Austin",
            "Austin;TX;USA": "Austin",
            "Dripping Springs TX": "Dripping Springs",
            "N Austin": "Austin"
            }

def get_element():
    
    """

    Reference:
    https://discussions.udacity.com/t/changing-attribute-value-in-xml/44575/6
    """
    context = ET.iterparse("Austin_split.osm", events=('start', ))
    _, root = next(context)
    for event,element in context:
        if element.tag == "node" or element.tag == 'way':
            for tag in element.iter("tag"):
                if tag.attrib['k'] == 'addr:city':
                    if tag.attrib['v'] in mapping.keys(): #if attrib['v'] matches the keys in mapping:
                        tag.set('v', mapping[tag.attrib['v']]) #replace keys by values in the xml

        yield element 
        root.clear()
    
SAMPLE_FILE = "first_Austin_edited.osm"
    
with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # this is where the output of yield is called on the 
    for i, element in enumerate(get_element()):
        output.write(ET.tostring(element, encoding='utf-8')) #for each yielded element, write its all attributes and child tags 

    output.write('</osm>')


## Revise of Problem 3 about street names

The nonstandard street names not only happen when 'k' value equals to 'addr:street', but also appear when 'k' equals to 'name' under 'way' tag. I will replace all words that match keys of the following mapping dictionary by the values of keys. So 'S 1st St, Suite 104' will be revised to 'South 1st Street, Suite 104'.

In [5]:
mapping = {"ave": "Avenue",
           "blvd": "Boulevard",
           "ct": "Court",
           "cv": "Cove", 
           "dr": "Drive",
           "dr.": "Drive",
           "ln": "Lane",
           "rd": "Road",
           "st": "Street",
           "st.": "Street",
           "pl": "Place",
           "trl": "Trail",
           "ps": "Pass",
           "pkwy": "Parkway",
           "s": "South",
           "n": "North",
           "e": "East",
           "w": "West",
           "fm": "Farm-to-Market Road",
           "hwy": "Highway",
           "ih": "Interstate Highway",
           "i": "Interstate Highway",
           "rr": "Ranch Road"
          }

def get_element():  # to yield every element that has been revised in the xml
    
    """

    Reference:
    https://discussions.udacity.com/t/changing-attribute-value-in-xml/44575/6
    """
    context = ET.iterparse("first_Austin_edited.osm", events=('start', ))
    _, root = next(context)
    for event,element in context:
        if element.tag == "node":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == 'addr:street':  
                    v_without_comma = tag.attrib['v'].replace(",", "") 
                    #change 'S 1st St, Suite 104' to 'S 1st St Suite 104' so that 'St' can be replaced smoothly
                    v_list = v_without_comma.split()
                    for item in v_list:
                        if item.lower() in mapping.keys(): #if any words of string mathes keys of mapping:
                            new_v = tag.attrib['v'].replace(item, mapping[item.lower()],1) #replace the kays by values
                            tag.set('v', new_v) #revise it in xml
                            
        elif element.tag == "way":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == 'name' or tag.attrib['k'] == 'addr:street': #two ways to give information of street names
                    v_without_comma = tag.attrib['v'].replace(",", "")
                    v_list = v_without_comma.split()
                    for item in v_list:
                        if item.lower() in mapping.keys():
                            new_v = tag.attrib['v'].replace(item, mapping[item.lower()],1)
                            tag.set('v', new_v)
                   


        yield element
        root.clear()
    
SAMPLE_FILE = "second_Austin_edited.osm"
    
with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # this is where the output of yield is called on the 
    for i, element in enumerate(get_element()):
        output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

## Import data from csv to database

In [6]:
#extract data from xml to csv

%run data.py      #exactly same code in Case Study Lesson 11

In [7]:
#create database file from csv
"""

    Reference:
    https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/7
"""

#part 1: Create Database
sqlite_file = 'Austin.db'
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS nodes''')
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
cur.execute('''DROP TABLE IF EXISTS ways''')
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()

#part 2: Create tables
cur.execute('''
    CREATE TABLE nodes(id INTEGER PRIMARY KEY, 
    user TEXT, 
    uid INTEGER,
    version TEXT,
    lat DOUBLE PRECISION,
    lon DOUBLE PRECISION,
    timestamp TIMESTAMP,
    changeset INTEGER)
''')

cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER references node (id), 
    key TEXT, 
    value TEXT,
    type TEXT)
''')


cur.execute('''
    CREATE TABLE ways(id INTEGER PRIMARY KEY, 
    user TEXT, 
    uid INTEGER,
    version TEXT,
    timestamp TIMESTAMP,
    changeset INTEGER)
''')


cur.execute('''
    CREATE TABLE ways_tags(id INTEGER references way (id), 
    key TEXT, 
    value TEXT,
    type TEXT)
''')

cur.execute('''
    CREATE TABLE ways_nodes(id INTEGER references way (id), 
    node_id INTEGER, 
    position INTEGER)
''')


conn.commit()

#part 3: Read the csv and insert data
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['lat'], i['lon'], i['timestamp'], \
              i['changeset']) for i in dr]
    
# insert the formatted data
cur.executemany("INSERT INTO nodes(id, user, uid, version, lat, lon, timestamp, changeset) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()
    
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['key'],i['value'].decode("utf-8"), i['type']) for i in dr]
    
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['timestamp'], i['changeset']) for i in dr]
    
cur.executemany("INSERT INTO ways(id, user, uid, version, timestamp, changeset) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()
    
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'],i['value'].decode("utf-8"), i['type']) for i in dr]
    
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()
    
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'],i['position']) for i in dr]
    
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()

## Overview of the Data

### File Size

In [8]:

"""

    Reference:
    https://discussions.udacity.com/t/display-files-and-their-sizes-in-directory/186741/13
"""   
    
dirpath = 'C:/Users/data expert/Desktop/Austin Project'

files_list = []
for path, dirs, files in os.walk(dirpath):
    files_list.extend([(filename, size(os.path.getsize(os.path.join(path, filename)))) for filename in files])

for filename, size in files_list:
    print '{:.<40s}: {:5s}'.format(filename,size)

Austin.db...............................: 60M  
Austin_split.osm........................: 112M 
data.py.................................: 7K   
first_Austin_edited.osm.................: 136M 
nodes.csv...............................: 47M  
nodes_tags.csv..........................: 1015K
OSM wrangling project - Austin, TX.ipynb: 25K  
sample.osm..............................: 11M  
schema.py...............................: 2K   
schema.pyc..............................: 1K   
second_Austin_edited.osm................: 158M 
ways.csv................................: 3M   
ways_nodes.csv..........................: 13M  
ways_tags.csv...........................: 5M   
OSM wrangling project - Austin, TX-checkpoint.ipynb: 25K  



### Number of unique users

In [9]:
from pprint import pprint

conn = sqlite3.connect('Austin.db')
cur = conn.cursor()
cur.execute('SELECT COUNT(DISTINCT(all_uid.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as all_uid')
results = cur.fetchall()
pprint (results)

[(901,)]


### Number of nodes

In [10]:

cur.execute('SELECT COUNT(*) FROM nodes')
results = cur.fetchone()
pprint (results)

(512771,)


### Number of ways

In [11]:

cur.execute('SELECT COUNT(*) FROM ways')
results = cur.fetchone()
pprint (results)

(54027,)


### List all cities

In [12]:
cur.execute("SELECT DISTINCT city_union.value FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as city_union \
WHERE city_union.key = 'city' \
GROUP BY city_union.value")
                

results = cur.fetchall()
pprint (results)

[(u'Austin',),
 (u'Bee Cave',),
 (u'Buda',),
 (u'Cedar Creek',),
 (u'Cedar Park',),
 (u'Creedmoor',),
 (u'Del Valle',),
 (u'Dripping Springs',),
 (u'Lakeway',),
 (u'Manchaca',),
 (u'Manor',),
 (u'Pflugerville',),
 (u'West Lake Hills',)]


### Top 3 cuisines

In [13]:
cur.execute("SELECT tags.value, COUNT(*) as Num FROM \
            (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as tags \
            WHERE tags.key = 'cuisine' \
            GROUP BY tags.value \
            ORDER BY Num DESC \
            LIMIT 3")
             
results = cur.fetchall()
pprint (results)

[(u'burger', 15), (u'pizza', 8), (u'sandwich', 7)]


### Top 3 leisure place

In [14]:
cur.execute("SELECT tags.value, COUNT(*) as Num FROM \
            (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as tags \
            WHERE tags.key = 'leisure' \
            GROUP BY tags.value \
            ORDER BY Num DESC \
            LIMIT 3")
             
results = cur.fetchall()
pprint (results)

[(u'pitch', 86), (u'park', 48), (u'playground', 12)]


## Additional Ideas

### reviewed = 'no' and 'name' not equal to 'name_1'

When I explore the 'ways_tags' dataset, I found a interesting 'k' value: 'reviewed', whoes values coule be either 'yes' and 'no'. I wonder if there is some relations between reviewed = 'no' and some problematic data like 'name' not equal to 'name_1'. Let's explore and prove it.

In [15]:
# create view that contains all distinct id of 'name' not equal to 'name_1'

cur.execute("CREATE VIEW name_name1_not_equal as\
            SELECT DISTINCT name_one.id, name_one.value, name.value \
            FROM (SELECT * FROM ways_tags WHERE ways_tags.key = 'name_1') as name_one \
            LEFT JOIN\
            (SELECT * FROM ways_tags WHERE ways_tags.key = 'name') as name \
            on name_one.id = name.id \
            WHERE name_one.value != name.value")

In [16]:
# count how many distinct id having 'name' not equal to 'name_1'
cur.execute("SELECT COUNT (*) FROM name_name1_not_equal ")
                     
results = cur.fetchall()
pprint (results)

[(67,)]


In [17]:
# create view that contains all distinct id of reviewed = 'no'

cur.execute("CREATE VIEW reviewed_is_no as\
            SELECT * FROM ways_tags WHERE ways_tags.key = 'reviewed' AND ways_tags.value = 'no'")

In [18]:
# count how many id that have both 'name' not equal to 'name_1' and reviewed = 'no'

cur.execute("SELECT COUNT (*) FROM name_name1_not_equal INNER JOIN reviewed_is_no on \
            name_name1_not_equal.id = reviewed_is_no.id ")
                     
results = cur.fetchall()
pprint (results)

[(57,)]


Let's see the cases when reviewed = 'yes' and 'name' not equal to 'name_1' in the following:

In [19]:
# create view that contains all distinct id of reviewed = 'yes'
cur.execute("CREATE VIEW reviewed_is_yes as\
            SELECT * FROM ways_tags WHERE ways_tags.key = 'reviewed' AND ways_tags.value = 'yes'")

In [20]:
# count how many id that have both 'name' not equal to 'name_1' and reviewed = 'yes'

cur.execute("SELECT COUNT (*) FROM name_name1_not_equal INNER JOIN reviewed_is_yes on \
            name_name1_not_equal.id = reviewed_is_yes.id ")
                     
results = cur.fetchall()
pprint (results)

[(0,)]


There are 67 cases that 'name' not equal to 'name_1', and 57 of them have that 'reviewed' = 'no'. The rest 10 cases don't show that 'reviewed' = 'yes' since we got 0 when count how many cases that have both 'name' not equal to 'name_1' and reviewed = 'yes'. Therefore I think we can improve the data quality by encouraging contributors to review data input when create it. In addition, we should also encourage contributors to create the 'reviewed' tag in case other contributors want to help review the data input.

## Conclusion

It looks that the tags 'node' and 'way' have different problems after exploration. The tags under the 'way' may have repeated or inconsistant attributes when 'k' includes 'tiger'. The 'node' tags have problems more like overabbreviated street names or nonstandard city name formats. We encourage contributors to double check data when the data comes from Tiger GPS and make notes of that in the attributes.

## Reference

[Udacity's home page](https://www.udacity.com)

[stackoverflow's home page](https://stackoverflow.com/)